In [2]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [3]:
import holoviews as hv
import hvplot.polars  # noqa
import matplotlib.pyplot as plt
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.features.scaling import scale_min_max
from src.features.transforming import map_by_group
from src.log_config import configure_logging
from src.plots.averages import (
    add_ci_to_averages,
    average_over_stimulus_seeds,
    calculate_max_crosscorr_lag_over_averages,
    plot_averages_with_ci,
)

configure_logging(
    ignore_libs=("Comm", "bokeh", "tornado", "matplotlib"),
)

pl.Config.set_tbl_rows(12)  # for 12 seeds
hv.output(widget_location="bottom", size=150)

In [4]:
db = DatabaseManager()

In [5]:
with db:
    df = db.get_table(
        "Merged_and_labeled_data", exclude_trials_with_measurement_problems=False
    )

In [6]:
df

trial_id,trial_number,participant_id,timestamp,temperature,rating,eda_raw,eda_tonic,eda_phasic,ppg_raw,ppg_ibi_shimmer,heart_rate,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil,pupil_tonic,brow_furrow,cheek_raise,mouth_open,upper_lip_raise,nose_wrinkle,normalized_timestamp,stimulus_seed,skin_patch,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,strictly_increasing_intervals_without_plateaus,plateau_intervals,prolonged_minima_intervals
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u16,u8,u16,u16,u16,u16,u16,u16,u16
1,1,1,294198.9762,0.0,0.425,0.743774,0.743503,0.000271,1408.404382,-1.062977,65.243881,5.670313,6.173421,5.641376,5.15944,5.400408,5.350094,0.00036,0.00286,0.0048,0.0,0.00007,0.0,396,1,0,0,1,0,0,0,0
1,1,1,294298.9762,0.000039,0.425,0.744641,0.743517,0.001124,1372.864366,-0.131651,65.242949,5.602895,6.142564,5.639922,5.157792,5.398857,5.343983,0.00036,0.00286,0.0048,0.0,0.00007,100.0,396,1,0,0,1,0,0,0,0
1,1,1,294398.9762,0.000154,0.395798,0.74488,0.743537,0.001343,1397.15563,-2.948082,65.241069,5.518026,6.072123,5.643301,5.160338,5.401819,5.337462,0.000341,0.002327,0.004199,0.0,0.000065,200.0,396,1,0,0,1,0,0,0,0
1,1,1,294498.9762,0.000417,0.270286,0.744808,0.743557,0.001252,1485.360361,1.488009,65.238629,5.399583,5.947853,5.638149,5.156922,5.397535,5.330919,0.000512,0.00122,0.002504,0.0,0.000065,300.0,396,1,0,0,1,0,0,0,0
1,1,1,294598.9762,0.000818,0.129521,0.74487,0.743578,0.001293,1496.64849,-4.735116,65.235505,5.276378,5.796069,5.646014,5.1618,5.403907,5.324177,0.001059,0.000701,0.002202,0.000006,0.000093,400.0,396,1,0,0,1,0,0,0,0
1,1,1,294698.9762,0.001358,0.152477,0.744957,0.743594,0.001363,1456.946658,1.28125,65.233139,5.158002,5.642459,5.62452,5.146081,5.3853,5.317484,0.001138,0.000565,0.001937,0.000017,0.00009,500.0,396,1,0,0,1,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
529,12,50,2.6956e6,0.031312,0.0,26.688608,26.716225,0.003169,1323.813266,-5.036073,59.279672,5.166699,4.766597,4.768156,5.167254,4.967705,5.180142,0.000465,0.001124,0.007081,0.00022,0.004168,179500.0,806,1,2645,1587,0,0,0,0,0
529,12,50,2.6957e6,0.030874,0.0,26.68499,26.716212,0.000389,1244.871959,44.337204,58.673362,5.026438,4.683481,4.690238,5.025632,4.857935,5.174284,0.000698,0.000846,0.007044,0.000348,0.004289,179600.0,806,1,2645,1587,0,0,0,0,0


In [7]:
scale_min_max(df).hvplot(
    x="timestamp",
    y=["pupil_l"],
    groupby=["participant_id", "trial_number"],
    muted_alpha=0,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'8c8fb177-f235-406f-ad57-b4c8a65f1505': {'version…

In [8]:
with db:
    raw = db.get_table("Raw_Pupil")
    process = db.get_table("Preprocess_Pupil")


In [9]:
raw

trial_id,trial_number,participant_id,rownumber,timestamp,pupil_l_raw,pupil_r_raw
u16,u8,u8,u32,f64,f64,f64
1,1,1,17631,294198.9762,5.73628,6.245389
1,1,1,17632,294215.605,5.726105,6.240875
1,1,1,17633,294232.4178,5.710716,6.233272
1,1,1,17634,294248.9778,5.703059,6.225731
1,1,1,17635,294265.7282,5.693711,6.221862
1,1,1,17636,294282.4034,5.679268,6.218964
…,…,…,…,…,…,…
529,12,50,161529,2.6960e6,4.940831,4.702713
529,12,50,161530,2.6961e6,4.942234,4.707606


In [10]:
raw.hvplot(
    x="timestamp",
    y=["pupil_l_raw", "pupil_r_raw"],
    groupby=["participant_id", "trial_number"],
    muted_alpha=0,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'56e67789-9fe2-4fe6-89ae-067df27b6fb9': {'version…

In [11]:
df.hvplot(
    x="timestamp",
    y=["pupil", "pupil_tonic"],
    groupby=["participant_id", "trial_number"],
    muted_alpha=0,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'874d6086-7be8-424b-ba8d-41b8a7dc4456': {'version…

In [ ]:
with db:
    raw = db.get_table("Raw_PPG")
    process = db.get_table("Preprocess_PPG")
    feature = db.get_table("Feature_PPG")


In [ ]:
raw.hvplot(
    x="timestamp",
    y=["ppg_heart_rate_shimmer"],
    groupby=["participant_id", "trial_number"],
    muted_alpha=0,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'1d7d15dc-867b-4bc3-b0d5-cf8bf930e9fe': {'version…

In [ ]:
feature.hvplot(
    x="timestamp",
    y=["heart_rate"],
    groupby=["participant_id", "trial_number"],
    muted_alpha=0,
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'51e37e40-dcfb-4cb9-a22b-fde046cb53fb': {'version…